# **sMArtYBEt**


## Betting Market Efficiency In Football


1. ##### Introduction


The world of sports betting is a multi-billion-dollar industry, a chaotic marketplace where passion for sport collides with the calculus of probability. For many, betting is driven by gut instinct and the thrill of the game. For bookmakers, it is a precisely engineered business, where odds are set not just to predict outcomes, but to balance books and ensure a profit regardless of the result. This profit, the "vig", is the market's built-in testament to their model and immanent inefficiency of the crowd. The prevailing wisdom suggest that for the average bettor, beating the bookmaker long-term is fool's errand- a game where the odds are permanently stacked against them.

It was against this public opinion that the idea for **sMArtyBEt** was born. I questioned this accepted idea by implementing a system that would approach the betting market not as a fan, but as an analyst. The project uses machine learning to independently forecast sports mach outcomes and indentify situation where its assessment of probability disagrees with the market's.



* Hypothesis

The idea is built upon the core hypothesis: *_Are the sports betting market efficient, or can a machine learning find a predictable patterns that the market has missed?*
The world of sports betting is often portrayed as a matter of luck and this project assumes that the market driven by public sentiment is inefficient, and it tends to prove that by finding these inaccuracies it not only can beat the market but turn it into disciplined science.

* Data

The foundational data was sourced from https://www.football-data.co.uk, providing comprehensive coverage of the football landscape. The dataset includes detailed match information with columns such as:
- Date: Match date (dd/mm/yy)
- Time: Kick-off time
- HomeTeam/AwayTeam: Team names
- FTHG/HG: Full Time Home Team Goals
- FTAG/AG: Full Time Away Team Goals

The complete data dictionary is available at https://www.football-data.co.uk/notes.txt. This analysis utilizes 9 seasons of historical English Premier League data from 2015 to 2024, focusing on a curated set of columns selected during data cleaning.

The final feature vector consists of 27 engineered features across three categories:
- Market-derived features: Implied probabilities for each outcome calculated from bookmaker odds after removing the vig
- Team strength features: Rolling averages for goals scored/conceded over a 10-match window, quantifying recent performance
- Quick-win features: Indicators capturing significant team advantages or favorable match con

* Problem formulation

_1. Multi-class classification_

At its core, the problem is structures as a multi-class classification task. The objective is to predict one of three possible outcomes for a given football match:

* Class 0 (A): Away Team Win
* Class 1 (D): Draw
* Class 2 (H): Home Team Win

Unlike standard classification problems where only the final label matters, the accuracy of the predicted probabilities is really significant. The model's output must be a well-calibrated probability distribution, where each probability reflects the true likelihood of that outcome. A model that is 90% confident in a win that only happens 70% of the time is poorly calibrated and will be unprofitable, even if it is accurate.

_2. Value betting_

A bet is placed only when a specific criterion is met, transforming the problem from prediction to value investing. The decision rule is defined as follows:

Edge(i) = Pmodel(i) - Pmarket(i) > threshold, where

Pmodel(i) - the model's predicted probability for outcome i,

 Pmarket(i) - the implied probability derived from the bookmaker's odds, after adjusting for the vig,

threshold - the minimum edge you require before placing a bet. It acts as a filter to ensure you only act on convictions strong enough to be profitable

A bet is triggered not when the model predicts an outcome, but when it is more confident than the betting market by a statistically significant margin. The magnitude of this edge directly quantifies the perceived market inefficiency.

_3. Evaluation Metrics_

The success of this project is evaluated using a different sets of metrics:

Model Performance Metrics:

* Accuracy

Financial Performance Metrics: To validate the core hypothesis.

* Return on Investment (ROI): The ultimate measure of profitability.
* Sharpe Ratio: To assess risk-adjusted returns, ensuring profits are not the result of excessive volatility.
* Maximum Drawdown: To see the worst-case peak-to-trough decline in the betting bankroll.

* ML model training

To forecast football match outcomes, i trained five different machine-learning models:

* Logistic Regression
* Polynomial Logistic Regression
* Random Forest Classifier
* XGBoost Classifier
* A simple Ensemble Model (averaging probabilities from the best-performing models)

Because football matches occur chronologically, time order must be preserved during training. Using a standard k-fold cross-validation would leak future information into the past, which artificially inflates the evaluation metrics. To avoid this data leakage, I used time series cross-validation. This way the model always predicts games it has never seen and could not have seen at training time. I experimented with several fold counts and found that 5 time-series folds provided the best balance between large training windows, sufficient test windows and low variance in accuracy estimates.

After computing fold-by-fold accuracies for all models, I averaged performance across all folds. The one with the best average accuracy is selected as the final predictive model. Once selected, this model was refit on the largest possible training window and then passed to calibrate_model method.
Since the model is already trained (cv='prefit'), the calibration step does not alter the classifier’s decision function.Instead, it learns a monotonic (method='isotonic') mapping from raw predicted probabilities to calibrated probabilities. This mapping is estimated on the last test block- a set of unseen data.
The resulting calibrated model is subsequently used for value-bet identification and for running the bankroll simulations.

* Bankroll simulations

To evaluate whether the calibrated predictive model can generate real betting profits, I implemented two bankroll simulation strategies: fixed-stake and fractional Kelly.

Fixed-stake:

 Each bet uses the same fixed amount (e.g., $100). This serves as a baseline that shows pure profitability without compounding effects or adaptive stake sizing.

Fractional Kelly:

The Kelly criterion determines the optimal fraction of the bankroll to bet based on the estimated edge. full_kelly = (p * (b + 1) - 1) / b p = model’s estimated probability of the outcome b = odds expressed as net profit f = optimal fraction of bankroll to wager Because full Kelly is too aggressive and leads to large bankroll swings, I use fractional Kelly (e.g., 0.25 × Full Kelly). This reduces risk while still adjusting stake sizes dynamically based on model confidence.

full_kelly = (p * (b + 1) - 1) / b, where

* p = model’s estimated probability of the outcome
* b = odds expressed as net profit per 1 unit staked
* full_kelly = optimal fraction of bankroll to wager

Because full Kelly is too aggressive and leads to large bankroll swings, I use fractional Kelly (e.g., 0.25 × Full Kelly). This reduces risk while still adjusting stake sizes dynamically based on model confidence.

* Sharpe Ratio

The Sharpe Ratio evaluates the risk-adjusted performance of a betting strategy. It measures how much excess return a strategy generates per unit of volatility (risk). A higher Sharpe Ratio indicates a more efficient strategy — one that provides better returns relative to the risk taken. The formula used in this project is: (mean_r - risk_free_rate / periods_per_year) / std_r * np.sqrt(periods_per_year) mean_r - the average periodic return of the strategy

(mean_r - risk_free_rate / periods_per_year) / std_r * np.sqrt(periods_per_year)

* mean_r - the average periodic return of the strategy
* std_r - standard deviation of returns(volatility)
* risk_free_rate - the guaranteed annual return of a zero-risk investment
* periods_per_year - number of return periods per year(252 is a standard trading-days approximation)
* np.sqrt(periods_per_year) - converts the Sharpe Ratio into an annualized metric

Annualizing ensures that strategies with different numbers of bets or time horizons can be compared fairly on a consistent yearly scale.